In [317]:
import pandas as pd

In [318]:
# Display all columns
pd.set_option("display.max_columns", None) 
# display all rows
pd.set_option("display.max_rows", None)

In [319]:
list_dataframe = []
for i in range(1, 13):
    #dataset/moroccan-houce-price-1.csv
    path = f"/home/marwane/mlops-projects/morocco-appartements-price/dataset/moroccan-immobilier-houses-price-{i}.csv"
    data = pd.read_csv(path)
    list_dataframe.append(data)

len(list_dataframe)

12

In [320]:
df = pd.concat(list_dataframe, axis=0)
df.head(2)

,price,surface,piece,rooms,bathroom,address,description,more_features_1_dict,more_features_2_dict
0,1 062 700 DH,112 m²,3 Pièces,2 Chambres,1 Salle de bain,Sidi Maarouf à Casablanca,Appartement à vendre à Sidi Maarouf. 2 chambre...,{'Type de bien\nAppartement': 'Type de bien\nA...,{}
1,913 100 DH,60 m²,1 Salle de bain,NaN,NaN,Hay Hassani à Casablanca,Plateau bureau à vendre en plein quartier Hay ...,{'Type de bien\nBureau': 'Type de bien\nBureau...,{}


In [321]:
def basic_analysis(df):
    print(f"=== Data shape:\n{df.shape}\n ")  
    print(f"=== Data Type:\n{df.dtypes}")  
    print(f"=== Data info:\n{df.info}") 
    print(f"=== Duplicated data:\n{df.duplicated().sum()}") 
    print(f"=== NaN values:\n{df.isna().sum()}") 

basic_analysis(df=df)

=== Data shape:
(8353, 9)
 
=== Data Type:
price                   object
surface                 object
piece                   object
rooms                   object
bathroom                object
address                 object
description             object
more_features_1_dict    object
more_features_2_dict    object
dtype: object
=== Data info:
<bound method DataFrame.info of                                        price            surface  \
0                               1 062 700 DH             112 m²   
1                                 913 100 DH              60 m²   
2                                 894 400 DH              71 m²   
3                              45 000 000 DH            2500 m²   
4                               2 500 000 DH             176 m²   
5                               1 159 160 DH             119 m²   
6                               1 458 000 DH             118 m²   
7                               1 212 000 DH              53 m²   
8             

The data  has 8353 rows and  9 columns, the data type of all columns is object as dtypes function return the need to be transformed. tha last two columns contains an a dictionaries as a values and each dictionary contain the athor features of each stock. first think to do is to transorm those columns into columns.


 There are also 159 duplicates rows and NaN values .....to be edited

### Dataset description

The data dataset was collected from web site called `mubawab`, I was scraped dat rlate to the appartements, scraping all features of each appartment. SO these dataset is real and it's a this project is very important to any one how want to dive in data scuance and  machine learning.

### Files

### Columns:

* surface: surface of appartment

* piece: number of (bathroom, bedroom, setroom and suisine)

* rooms: number of rooms in the house

* bathroom: number of bathroom in house

* address: address of house whre is located

* more_infos: precise infos about house

* description: explaination of house and it's pieces

* feat_infos_dic1: dictionary of main features

* feat_infos_dic2: dictionary of other features


For the last features that contains a dictionaries as a values needed more preparation to be transform each featurs as a columns, the feature feat_infos_dic1 has a lot of features and each one of them hase labels, for the feat_infos_dic2 contain also a lot features but are binary.

### Target

* price: the price of appartment and it's a target

## Function that transform columns contains dictionaries as values into columns

In [322]:
# Function to clean the keys (remove text after \n)
def clean_key(key, value):
    """Methode that remove key and conserve the value"""
    return key.split('\n')[0], value.split("\n")[1] 

In [323]:
import ast
 
def object_dict_to_dataframe(df):
    """Methode that transform columns with dictionaries as a values into columns as features"""
    # The string representation of the dictionary
    dict_str1 = df["more_features_1_dict"].values.tolist()
    list_dict1 = []
    # Convert the string into a dictionary
    for i in range(len(dict_str1)):
        dict_obj = ast.literal_eval(dict_str1[i])
        list_dict1.append(dict_obj)

    # Apply the clean_key function to each dictionary's keys
    cleaned_dict_list1 = [{clean_key(k, v)[0]: clean_key(k, v)[1] for k, v in zip(d.keys(), d.values())} for d in list_dict1]
    df['dict1'] = cleaned_dict_list1 

    # Convert the list of dictionaries into a DataFrame
    df_dict1 = pd.DataFrame(df.dict1)

    # Assuming the column we want to convert is the entire DataFrame itself (as it contains dicts)
    # We can expand the dictionaries to separate columns
    expanded_df_dict1 = pd.json_normalize(df_dict1.to_dict(orient='records'))

    # ========================================================= #
    dict_str2 = df["more_features_2_dict"].values.tolist()
    list_dict2 = []  
    # Convert the string into a dictionary
    for i in range(len(dict_str2)):
        dict_obj = ast.literal_eval(dict_str2[i])
        list_dict2.append(dict_obj)

    df['dict2'] = list_dict2 

    # Convert the list of dictionaries into a DataFrame
    df_dict2 = pd.DataFrame(df.dict2) 
    # Assuming the column we want to convert is the entire DataFrame itself (as it contains dicts)
    # We can expand the dictionaries to separate columns
    expanded_df_dict2 = pd.json_normalize(df_dict2.to_dict(orient='records'))

    df_main_feat = df[['price', 'surface', 'piece', 'rooms', 'bathroom', 'address', 'description']].copy()

    concat_df = pd.concat([df_main_feat.reset_index(drop=True), expanded_df_dict1, expanded_df_dict2], axis=1)

    return concat_df


data = object_dict_to_dataframe(df)
# Display the expanded DataFrame
print(data.shape) 

(8353, 65)


## Edit columns name

In [324]:
# format features

def format_columns_name(df):
    """Methode that edit columns names"""
    new_colums_name = [string.lower().replace(" ", "_").replace("dict2_", "").replace("-", "_").replace(".", "_") for string in df.columns.to_list()]
    dic_ond_new_features = {}
    for old_name, new_name in zip(df.columns, new_colums_name):
        dic_ond_new_features.update({old_name: new_name})
 
    df = df.rename(columns=dic_ond_new_features) 
    
    # format features
    new_colums_name = [string.lower().replace(" ", "_").replace("dict1_", "").replace(":", "1").replace("-", "_").replace(".", "_").replace("+_", "").replace("d'", "") for string in df.columns.to_list()]
    dic_ond_new_features = {}
    for old_name, new_name in zip(df.columns, new_colums_name):
        dic_ond_new_features.update({old_name: new_name}) 

    df = df.rename(columns=dic_ond_new_features) 

    return df 

data = format_columns_name(data)
data.head(2)

,price,surface,piece,rooms,bathroom,address,description,type_de_bien,etat,état,étage_du_bien,standing,livraison1,etat_du_bien,orientation,type_du_sol,surface_habitable,surface_extérieure,type_de_terrain,constructibilité,livraison,statut_du_terrain,surface_de_la_parcelle,nombre_étages,dict2_ascenseur,dict2_sécurité,dict2_cuisine_équipée,dict2_four,dict2_jardin,dict2_terrasse,dict2_garage,dict2_piscine,dict2_salon_marocain,dict2_salon_européen,dict2_antenne_parabolique,dict2_chauffage_central,dict2_concierge,dict2_cheminée,dict2_climatisation,dict2_double_vitrage,dict2_porte_blindée,dict2_réfrigérateur,dict2_machine_à_laver,dict2_micro_ondes,dict2_chambre_rangement,dict2_façade_extérieure,dict2_vue_sur_mer,dict2_meublé,dict2_3_extras,dict2_7_extras,dict2_6_extras,dict2_vue_sur_les_montagnes,dict2_9_extras,dict2_4_extras,dict2_17_extras,dict2_5_extras,dict2_2_extras,dict2_entre_seul,dict2_1_extra,dict2_13_extras,dict2_11_extras,dict2_16_extras,dict2_10_extras,dict2_8_extras,dict2_12_extras
0,1 062 700 DH,112 m²,3 Pièces,2 Chambres,1 Salle de bain,Sidi Maarouf à Casablanca,Appartement à vendre à Sidi Maarouf. 2 chambre...,Appartement,Nouveau,Finalisé,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,913 100 DH,60 m²,1 Salle de bain,NaN,NaN,Hay Hassani à Casablanca,Plateau bureau à vendre en plein quartier Hay ...,Bureau,Nouveau,En cours de construction,8ème,Moyen standing,août 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [325]:
#data = data.drop([col_name for col_name in data.columns if "dict2" in col_name], axis=1)
data.columns

Index(['price', 'surface', 'piece', 'rooms', 'bathroom', 'address',
       'description', 'type_de_bien', 'etat', 'état', 'étage_du_bien',
       'standing', 'livraison1', 'etat_du_bien', 'orientation', 'type_du_sol',
       'surface_habitable', 'surface_extérieure', 'type_de_terrain',
       'constructibilité', 'livraison', 'statut_du_terrain',
       'surface_de_la_parcelle', 'nombre_étages', 'dict2_ascenseur',
       'dict2_sécurité', 'dict2_cuisine_équipée', 'dict2_four', 'dict2_jardin',
       'dict2_terrasse', 'dict2_garage', 'dict2_piscine',
       'dict2_salon_marocain', 'dict2_salon_européen',
       'dict2_antenne_parabolique', 'dict2_chauffage_central',
       'dict2_concierge', 'dict2_cheminée', 'dict2_climatisation',
       'dict2_double_vitrage', 'dict2_porte_blindée', 'dict2_réfrigérateur',
       'dict2_machine_à_laver', 'dict2_micro_ondes', 'dict2_chambre_rangement',
       'dict2_façade_extérieure', 'dict2_vue_sur_mer', 'dict2_meublé',
       'dict2_3_extras', 'dict2_

In [326]:
dd = data.T
dd = dd.drop_duplicates().T
dd.head(2)

,price,surface,piece,rooms,bathroom,address,description,type_de_bien,etat,état,étage_du_bien,standing,livraison1,etat_du_bien,orientation,type_du_sol,surface_habitable,surface_extérieure,type_de_terrain,constructibilité,livraison,statut_du_terrain,surface_de_la_parcelle,nombre_étages,dict2_ascenseur,dict2_sécurité,dict2_cuisine_équipée,dict2_four,dict2_jardin,dict2_terrasse,dict2_garage,dict2_piscine,dict2_salon_marocain,dict2_salon_européen,dict2_antenne_parabolique,dict2_chauffage_central,dict2_concierge,dict2_cheminée,dict2_climatisation,dict2_double_vitrage,dict2_porte_blindée,dict2_réfrigérateur,dict2_machine_à_laver,dict2_micro_ondes,dict2_chambre_rangement,dict2_façade_extérieure,dict2_vue_sur_mer,dict2_meublé,dict2_3_extras,dict2_7_extras,dict2_6_extras,dict2_vue_sur_les_montagnes,dict2_9_extras,dict2_4_extras,dict2_17_extras,dict2_5_extras,dict2_2_extras,dict2_entre_seul,dict2_1_extra,dict2_13_extras,dict2_11_extras,dict2_16_extras,dict2_10_extras,dict2_8_extras,dict2_12_extras
0,1 062 700 DH,112 m²,3 Pièces,2 Chambres,1 Salle de bain,Sidi Maarouf à Casablanca,Appartement à vendre à Sidi Maarouf. 2 chambre...,Appartement,Nouveau,Finalisé,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,913 100 DH,60 m²,1 Salle de bain,NaN,NaN,Hay Hassani à Casablanca,Plateau bureau à vendre en plein quartier Hay ...,Bureau,Nouveau,En cours de construction,8ème,Moyen standing,août 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [327]:
data.columns

Index(['price', 'surface', 'piece', 'rooms', 'bathroom', 'address',
       'description', 'type_de_bien', 'etat', 'état', 'étage_du_bien',
       'standing', 'livraison1', 'etat_du_bien', 'orientation', 'type_du_sol',
       'surface_habitable', 'surface_extérieure', 'type_de_terrain',
       'constructibilité', 'livraison', 'statut_du_terrain',
       'surface_de_la_parcelle', 'nombre_étages', 'dict2_ascenseur',
       'dict2_sécurité', 'dict2_cuisine_équipée', 'dict2_four', 'dict2_jardin',
       'dict2_terrasse', 'dict2_garage', 'dict2_piscine',
       'dict2_salon_marocain', 'dict2_salon_européen',
       'dict2_antenne_parabolique', 'dict2_chauffage_central',
       'dict2_concierge', 'dict2_cheminée', 'dict2_climatisation',
       'dict2_double_vitrage', 'dict2_porte_blindée', 'dict2_réfrigérateur',
       'dict2_machine_à_laver', 'dict2_micro_ondes', 'dict2_chambre_rangement',
       'dict2_façade_extérieure', 'dict2_vue_sur_mer', 'dict2_meublé',
       'dict2_3_extras', 'dict2_

In [328]:
dd.shape

(8353, 65)

In [329]:
data.to_csv("data-houses-price-orgnised-not-prepared.csv", index=False)

In [330]:
data.columns

Index(['price', 'surface', 'piece', 'rooms', 'bathroom', 'address',
       'description', 'type_de_bien', 'etat', 'état', 'étage_du_bien',
       'standing', 'livraison1', 'etat_du_bien', 'orientation', 'type_du_sol',
       'surface_habitable', 'surface_extérieure', 'type_de_terrain',
       'constructibilité', 'livraison', 'statut_du_terrain',
       'surface_de_la_parcelle', 'nombre_étages', 'dict2_ascenseur',
       'dict2_sécurité', 'dict2_cuisine_équipée', 'dict2_four', 'dict2_jardin',
       'dict2_terrasse', 'dict2_garage', 'dict2_piscine',
       'dict2_salon_marocain', 'dict2_salon_européen',
       'dict2_antenne_parabolique', 'dict2_chauffage_central',
       'dict2_concierge', 'dict2_cheminée', 'dict2_climatisation',
       'dict2_double_vitrage', 'dict2_porte_blindée', 'dict2_réfrigérateur',
       'dict2_machine_à_laver', 'dict2_micro_ondes', 'dict2_chambre_rangement',
       'dict2_façade_extérieure', 'dict2_vue_sur_mer', 'dict2_meublé',
       'dict2_3_extras', 'dict2_

In [331]:
data.head()

,price,surface,piece,rooms,bathroom,address,description,type_de_bien,etat,état,étage_du_bien,standing,livraison1,etat_du_bien,orientation,type_du_sol,surface_habitable,surface_extérieure,type_de_terrain,constructibilité,livraison,statut_du_terrain,surface_de_la_parcelle,nombre_étages,dict2_ascenseur,dict2_sécurité,dict2_cuisine_équipée,dict2_four,dict2_jardin,dict2_terrasse,dict2_garage,dict2_piscine,dict2_salon_marocain,dict2_salon_européen,dict2_antenne_parabolique,dict2_chauffage_central,dict2_concierge,dict2_cheminée,dict2_climatisation,dict2_double_vitrage,dict2_porte_blindée,dict2_réfrigérateur,dict2_machine_à_laver,dict2_micro_ondes,dict2_chambre_rangement,dict2_façade_extérieure,dict2_vue_sur_mer,dict2_meublé,dict2_3_extras,dict2_7_extras,dict2_6_extras,dict2_vue_sur_les_montagnes,dict2_9_extras,dict2_4_extras,dict2_17_extras,dict2_5_extras,dict2_2_extras,dict2_entre_seul,dict2_1_extra,dict2_13_extras,dict2_11_extras,dict2_16_extras,dict2_10_extras,dict2_8_extras,dict2_12_extras
0,1 062 700 DH,112 m²,3 Pièces,2 Chambres,1 Salle de bain,Sidi Maarouf à Casablanca,Appartement à vendre à Sidi Maarouf. 2 chambre...,Appartement,Nouveau,Finalisé,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,913 100 DH,60 m²,1 Salle de bain,NaN,NaN,Hay Hassani à Casablanca,Plateau bureau à vendre en plein quartier Hay ...,Bureau,Nouveau,En cours de construction,8ème,Moyen standing,août 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,894 400 DH,71 m²,3 Pièces,2 Chambres,2 Salles de bains,Aïn Sebaâ à Casablanca,Très bel appartement à vendre à Ain Sebaa\nCha...,Appartement,Nouveau,Finalisé,NaN,Moyen standing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ascenseur,Sécurité,Cuisine équipée,Four,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,45 000 000 DH,2500 m²,7 Pièces,5 Chambres,4 Salles de bains,Anfa Supérieur à Casablanca,Villa à démolir à vendre sur Anfa Supérieur\nV...,Villa,À rénover,NaN,NaN,NaN,NaN,Moins d'un an,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sécurité,NaN,NaN,Jardin,Terrasse,Garage,Piscine,Salon Marocain,Salon européen,Antenne parabolique,Chauffage central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2 500 000 DH,176 m²,5 Pièces,3 Chambres,3 Salles de bains,Mers Sultan à Casablanca,Un Magnifique Appartement a vendre sur Casabla...,Appartement,Bon état,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ascenseur,NaN,Cuisine équipée,Four,Jardin,Terrasse,Garage,NaN,NaN,NaN,NaN,NaN,Concierge,Cheminée,Climatisation,Double vitrage,Porte blindée,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## ===================== END =========================